In [1]:
from iexfinance.stocks import Stock
import requests
import urllib.parse
import requests
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import time

In [3]:
symbol = "ADBE"

In [4]:
period = 'quarter'
time = '12'

In [5]:
BASE_URL = 'https://cloud.iexapis.com/stable/'
endpoint = f"stock/{symbol}/financials/"

In [6]:
url = urllib.parse.urljoin(BASE_URL, endpoint)

In [7]:
#Query String Parameters: token=YOUR_TOKEN_HERE&period={} 

In [8]:
url

'https://cloud.iexapis.com/stable/stock/ADBE/financials/'

In [ ]:
import os
token = os.environ['SANDBOX_TOKEN']

In [9]:
adbe = pd.DataFrame(requests.get(f'https://sandbox.iexapis.com/stable/time-series/REPORTED_FINANCIALS/ADBE/10-K?from=2010-01-01&limit=10&token={token}').json())

In [10]:
adbe.head()

,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccruedIncomeTaxesCurrent,AccruedLiabilitiesCurrent,AccruedRestructuringExpensesCurrent,AccruedRestructuringExpensesNoncurrent,AccumulatedOtherComprehensiveIncomeLossNetOfTax,AdditionalPaidInCapital,AdjustmentToValuationOfAssumedOptions,AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesOther,...,TaxCutsandJobsActonetimetransitiontax,TaxchargeduetoTaxCutsandJobsAct,TransitiontaxexpenseoflongtermincometaxespayableduetotheTaxCutsandJobsAct,TransitiontaxexpenseondeferredforeignearningsduetoTaxCutsandJobsAct,TransitiontaxexpenseonothertaxliabilitiesduetoTaxCutsandJobsAct,TreasuryStockSharesAcquired,USFederalstatutorytaxrateeffectiveaftertheTaxCutsandJobsAct,USFederalstatutorytaxrateeffectivebeforetheTaxCutsandJobsAct,UnrealizedGainsonAvailableforSaleSecuritiesNetofTax,UnrealizedLossesonAvailableforSaleSecuritiesNetofTax
0,60140339,412823480,48427154,430514810,38800255.0,6936183.0,24934912,2483714665,-30066646.0,4437634.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54882381,563540701,53881183,565123108,8475796.0,8360685.0,17796956,2500334409,NaN,3369101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,87952043,662825031,42845702,569184330,NaN,NaN,31152029,2873678462,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51499503,635013573,51428664,604635210,NaN,NaN,31849945,3065756649,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63630853,605923925,10304919,683625981,NaN,NaN,47213738,3483932020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
## Add quarter
adbe['quarter'] = adbe['periodEnd'].apply(lambda x: time.strftime('%m/%Y',  time.gmtime(x/1000)))

AttributeError: 'str' object has no attribute 'strftime'

In [12]:
income_json = f'https://sandbox.iexapis.com/stable/stock/{symbol}/income?period={period}&last={time}&token={token}'
cash_json = f'https://sandbox.iexapis.com/stable/stock/{symbol}/cash-flow?period={period}&last={time}&token={token}'
balance_json = f'https://sandbox.iexapis.com/stable/stock/{symbol}/balance-sheet?period={period}&last={time}&token={token}'
financials_json = f"https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period={period}&last={time}&token={token}"
fundamentals = f"https://sandbox.iexapis.com/stable/time-series/fundamentals/{symbol}/{period}ly?range={time}q&token={token}"

In [14]:
income = pd.DataFrame(requests.get(income_json).json()['income'])
cash = pd.DataFrame(requests.get(cash_json).json()['cashflow'])
balance = pd.DataFrame(requests.get(balance_json).json()['balancesheet'])

In [16]:
income = income.set_index(pd.to_datetime(income['fiscalDate'].apply(lambda x: x[:7])))
cash = cash.set_index(pd.to_datetime(cash['fiscalDate'].apply(lambda x: x[:7])))
balance = balance.set_index(pd.to_datetime(balance['fiscalDate'].apply(lambda x: x[:7])))

In [17]:
df = income.merge(cash, left_index=True, right_index=True, suffixes=('_i', '_c'))
df = df.merge(balance, left_index=True, right_index=True, suffixes=('', '_b'))
df = df.rename(str.lower, axis='columns')

In [18]:
df['financial_leverage'] = df['shareholderequity'] / df['totalassets']
df['debt_to_equity'] = df['totalliabilities'] / df['shareholderequity']
df['current'] = df['currentassets'] / df['totalcurrentliabilities']
df['quick'] = (df['currentassets'] + df['inventory']) / df['totalcurrentliabilities']

In [19]:
df['netmargin'] = df['netincome_c'] / df['totalrevenue']
df['asset_turnover'] = df['totalrevenue'] / df['totalassets']
df['cash_operations'] = df['netincome_c'] / df['totalrevenue']
df['roa'] = df['cash_operations'] * df['asset_turnover']
df['roe'] = df['netmargin'] * df['asset_turnover'] * df['financial_leverage']
df['operating_margin'] = df['cashflow'] / df['totalrevenue']
df["fcf"] = df['cashflow'] + df['capitalexpenditures']
df['levered_fcf'] = df['fcf'] / df['totalrevenue']
df['overhead/revenue'] = df['sellinggeneralandadmin'] / df['totalrevenue']

In [20]:
df[['netmargin','asset_turnover','cash_operations','roa','roe','operating_margin',"fcf",'levered_fcf','overhead/revenue','financial_leverage','debt_to_equity','current','quick']]

,netmargin,asset_turnover,cash_operations,roa,roe,operating_margin,fcf,levered_fcf,overhead/revenue,financial_leverage,debt_to_equity,current,quick
fiscalDate,,,,,,,,,,,,,
2020-11-01,0.674051,0.136643,0.674051,0.092104,0.050638,1.677221,5322696675,1.549633,0.361273,0.549787,0.798120,1.433542,1.433542
2020-08-01,0.296759,0.143437,0.296759,0.042566,0.022426,1.258315,3748419087,1.160216,0.361017,0.526849,0.940217,1.452680,1.452680
2020-05-01,0.347720,0.148267,0.347720,0.051556,0.026940,0.000810,2422546,0.000749,0.349796,0.522549,0.949063,1.263474,1.263474
2020-02-01,0.312060,0.149076,0.312060,0.046521,0.023406,0.000436,1289005,0.000406,0.358057,0.503134,1.048373,1.215088,1.215088
2019-11-01,0.280158,0.141663,0.280158,0.039688,0.019369,1.501506,4218873746,1.370340,0.340121,0.488018,1.005252,0.792521,0.792521
2019-08-01,0.281649,0.137579,0.281649,0.038749,0.019137,1.160445,3037840367,1.056699,0.374748,0.493873,0.955530,0.755410,0.755410
2019-05-01,0.225639,0.140871,0.225639,0.031786,0.015954,0.805079,2078408269,0.734785,0.389800,0.501921,0.996810,0.714257,0.714257
2019-02-01,0.253213,0.136519,0.253213,0.034568,0.017891,0.434301,1060077045,0.397198,0.374082,0.517561,0.976143,0.976044,0.976044
2018-11-01,0.278735,0.129185,0.278735,0.036008,0.018258,1.652009,3833669882,1.543576,0.381222,0.507045,0.999746,1.140674,1.140674


In [21]:
df['capitalexpenditures']

fiscalDate
2020-11-01   -438239042
2020-08-01   -316937195
2020-05-01      -197837
2020-02-01       -97498
2019-11-01   -403821592
2019-08-01   -298251100
2019-05-01   -198832010
2019-02-01    -99024205
2018-11-01   -269308183
2018-08-01   -202129112
2018-05-01   -135057436
2018-02-01    -66748601
Name: capitalexpenditures, dtype: int64

In [22]:
 url_adv_f = f"https://sandbox.iexapis.com/stable/time-series/fundamentals/{symbol}/annual?last=4&token=Tsk_910a8ace8ad14af698bc4a65be1a4219"

In [23]:
adbe_adv = pd.DataFrame(requests.get(url_adv_f).json())

In [24]:
adbe_adv

,accountsPayable,accountsPayableTurnover,accountsReceivable,accountsReceivableTurnover,asOfDate,assetsCurrentCash,assetsCurrentCashRestricted,assetsCurrentDeferredCompensation,assetsCurrentDeferredTax,assetsCurrentDiscontinuedOperations,...,stockTreasury,symbol,wabso,wabsoSplitAdjusted,wadso,wadsoSplitAdjusted,id,key,subkey,updated
0,307607173,42.655703,1467207664,9.631744,2021-01-15,6130588051,0,0,0,0,...,-13680440001,ADBE,488927628,500449721,507436818,505840503,MDUSLTNFEAAN,BDEA,alaunn,1681991188785
1,213116931,54.803361,1539164618,7.462859,2021-01-04,4323526636,0,0,0,0,...,-10809322748,ADBE,495404409,508474613,502044074,495238504,NUMENTLDAASF,ABED,nualna,1628835952386
2,195125159,48.986374,1376961114,6.866186,2021-01-16,3248551660,0,0,0,0,...,-8323185982,ADBE,499605480,494189000,514045036,519800535,UNLASEDMNFTA,EABD,luanna,1678085968510
3,114893587,65.449555,1263375740,6.281497,2021-01-05,6006845212,0,0,0,0,...,-6091316383,ADBE,508964648,505884243,503235073,516756304,LNSUANTEFMDA,DEAB,aunnla,1615782895178


In [25]:
url_adv_s = f"https://sandbox.iexapis.com/stable/stock/{symbol}/advanced-stats?token=Tsk_910a8ace8ad14af698bc4a65be1a4219"

In [26]:
adbe_adv_s = requests.get(url_adv_s).json()

In [27]:
adbe_financ = requests.get(financials_json).json()

In [28]:
df2 = pd.DataFrame(adbe_financ['financials'])
df2 = df2.set_index(pd.to_datetime(income['fiscalDate'].apply(lambda x: x[:7])))
df2.columns

Index(['EBITDA', 'accountsPayable', 'capitalSurplus', 'cashChange', 'cashFlow',
       'cashFlowFinancing', 'changesInInventories', 'changesInReceivables',
       'commonStock', 'costOfRevenue', 'currency', 'currentAssets',
       'currentCash', 'currentDebt', 'currentLongTermDebt', 'depreciation',
       'dividendsPaid', 'ebit', 'exchangeRateEffect', 'filingType',
       'fiscalDate', 'fiscalQuarter', 'fiscalYear', 'goodwill', 'grossProfit',
       'incomeTax', 'intangibleAssets', 'interestIncome', 'inventory',
       'investingActivityOther', 'investments', 'longTermDebt',
       'longTermInvestments', 'minorityInterest', 'netBorrowings', 'netIncome',
       'netIncomeBasic', 'netTangibleAssets', 'operatingExpense',
       'operatingIncome', 'operatingRevenue', 'otherAssets',
       'otherCurrentAssets', 'otherCurrentLiabilities',
       'otherIncomeExpenseNet', 'otherLiabilities', 'pretaxIncome',
       'propertyPlantEquipment', 'receivables', 'reportDate',
       'researchAndDevelo

In [29]:
df2['operating_margin'] = df2['operatingIncome'] / df2['revenue']

In [30]:
df2['net_margin'] = df2['netIncome'] / df2['revenue']

In [31]:
# to check cashflow
df2['fcf_margin'] = (df2['cashFlow'] + df['capitalexpenditures'])/df2['revenue']

In [32]:
df2['gross_margin'] = df['grossprofit'] / df['totalrevenue']

In [35]:
df2['roa'] = df2['net_margin']*df2['asset_turnover']

In [34]:
df2['asset_turnover'] = df2['revenue'] / df2['totalAssets']

In [38]:
df2['roa2'] = df2['netIncome']*4/ df2['totalAssets']

In [39]:
df2['roa2']

fiscalDate
2020-11-01    0.371602
2020-08-01    0.171631
2020-05-01    0.200528
2020-02-01    0.188246
2019-11-01    0.162496
2019-08-01    0.156825
2019-05-01    0.126457
2019-02-01    0.136545
2018-11-01    0.145514
2018-08-01    0.174865
2018-05-01    0.175423
2018-02-01    0.154868
Name: roa2, dtype: float64

In [36]:
df2['roa']

fiscalDate
2020-11-01    0.092900
2020-08-01    0.042908
2020-05-01    0.050132
2020-02-01    0.047062
2019-11-01    0.040624
2019-08-01    0.039206
2019-05-01    0.031614
2019-02-01    0.034136
2018-11-01    0.036378
2018-08-01    0.043716
2018-05-01    0.043856
2018-02-01    0.038717
Name: roa, dtype: float64

In [41]:
df2['shareholderEquity']


fiscalDate
2020-11-01    13394789540
2020-08-01    12109048882
2020-05-01    11304485286
2020-02-01    10691072029
2019-11-01    10631819329
2019-08-01    10685686032
2019-05-01    10410669560
2019-02-01     9926305441
2018-11-01     9503172309
2018-08-01     9209416777
2018-05-01     8851040588
2018-02-01     8842211862
Name: shareholderEquity, dtype: int64

In [49]:
for i in df.columns:
    if 'eb' in i:
        print(i)

ebit
netincomebasic
currentlongtermdebt
longtermdebt
debt_to_equity


In [43]:
df2.columns

Index(['EBITDA', 'accountsPayable', 'capitalSurplus', 'cashChange', 'cashFlow',
       'cashFlowFinancing', 'changesInInventories', 'changesInReceivables',
       'commonStock', 'costOfRevenue', 'currency', 'currentAssets',
       'currentCash', 'currentDebt', 'currentLongTermDebt', 'depreciation',
       'dividendsPaid', 'ebit', 'exchangeRateEffect', 'filingType',
       'fiscalDate', 'fiscalQuarter', 'fiscalYear', 'goodwill', 'grossProfit',
       'incomeTax', 'intangibleAssets', 'interestIncome', 'inventory',
       'investingActivityOther', 'investments', 'longTermDebt',
       'longTermInvestments', 'minorityInterest', 'netBorrowings', 'netIncome',
       'netIncomeBasic', 'netTangibleAssets', 'operatingExpense',
       'operatingIncome', 'operatingRevenue', 'otherAssets',
       'otherCurrentAssets', 'otherCurrentLiabilities',
       'otherIncomeExpenseNet', 'otherLiabilities', 'pretaxIncome',
       'propertyPlantEquipment', 'receivables', 'reportDate',
       'researchAndDevelo

In [46]:
df2['totalDebt']

fiscalDate
2020-11-01    4228277617
2020-08-01    4278214251
2020-05-01    4181773561
2020-02-01    4116441571
2019-11-01    4314586902
2019-08-01    4188426186
2019-05-01    4300986509
2019-02-01    4194630093
2018-11-01    4201899168
2018-08-01    1895973446
2018-05-01    1961342042
2018-02-01    1934380945
Name: totalDebt, dtype: int64

In [53]:
stats = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={token}"
stats = requests.get(stats).json()

In [54]:
stats

{'companyName': 'Adobe Inc',
 'marketcap': 224651357654,
 'week52high': 548.6,
 'week52low': 297,
 'week52change': 0.32108846868513014,
 'sharesOutstanding': 482000509,
 'float': 0,
 'avg10Volume': 3043332,
 'avg30Volume': 3114066,
 'day200MovingAvg': 496.19,
 'day50MovingAvg': 489.19,
 'employees': 0,
 'ttmEPS': 11.27,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '0',
 'exDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 42.174471565229716,
 'beta': 1.1450238559620052,
 'maxChangePercent': 10.265524085532302,
 'year5ChangePercent': 4.322443922106131,
 'year2ChangePercent': 0.8728570942862748,
 'year1ChangePercent': 0.31976415106979145,
 'ytdChangePercent': -0.082733940978197,
 'month6ChangePercent': 0.03258476132389889,
 'month3ChangePercent': 0.026969699524550854,
 'month1ChangePercent': -0.08639988566142091,
 'day30ChangePercent': -0.08294205652923639,
 'day5ChangePercent': -0.03033756362195382}

In [55]:
# Financial strength
financial_strength = pd.DataFrame()
financial_strength['cash_debt'] = financials['totalcash'] / financials['totaldebt']
financial_strength['equity_asset'] = financials['shareholderequity'] / financials['totalassets']
financial_strength['debt_equity'] = financials['totaldebt'] / financials['shareholderequity']
financial_strength['debt_ebitda'] = financials['totaldebt'] / (financials['ebitda']*4)

NameError: name 'financials' is not defined

In [56]:
# Profitability
profitability = pd.DataFrame()
profitability['operating_margin'] = financials['operatingincome'] / financials['revenue']
profitability['net_margin'] = financials['netincome'] / financials['revenue']
profitability['asset_turnover'] = financials['revenue'] / financials['totalassets']
profitability['roa'] = financials['netincome']*4/ financials['totalassets']
profitability['equity_multipl'] = financials['totalassets'] / financials['shareholderequity']
profitability['roe'] = financials['netincome'] / financials['shareholderequity']
#profitability['gross_margin'] = income['grossprofit'] / income['totalrevenue']

NameError: name 'financials' is not defined

In [ ]:
#def add_profitability(df):
    #df['operating_margin'] = df['operatingincome'] / df['revenue']
    #df['net_margin'] = df['netincome'] / df['revenue']
    #if df['time'] == 'y':
        #df['asset_turnover'] = df['revenue'] / collection_y['totalassets'].rolling(2).mean()